## Part 1 of a 3-part assignment for Week 3 on Toronto Neighborhood
- #### 1. Installations
- #### 2. Use a scaper from the internet
- #### 3. Wrangle the data, sort, delete etc.
- #### 4. Limitation: The difficulty has been in combining the Regent Park and Harborfront, as it will the latter one. [..........href="/wiki/Regent_Park" title="Regent Park">Harbourfront.........]

#### Import libraries and install Beautifulsoup4

In [1]:
import pandas as pd
import sys
sys.path.append('C:\\Python27\\Scripts\\pip.exe')
!pip install beautifulsoup4

!pip install lxml
!pip install html5lib
!pip install requests

from bs4 import BeautifulSoup
import requests
import urllib.request
from lxml import html

     |████████████████████████████████| 112kB 25.3MB/s eta 0:00:01
     |████████████████████████████████| 5.8MB 30.6MB/s eta 0:00:01


#### Scrape the web page containing Canadian postal codes, boroughs and neighborhoods

In [2]:
# Scrape the page with BeautifulSoup

quote_page='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

page = requests.get(quote_page)
#print(page.status_code)

soup= BeautifulSoup(page.text, "html.parser")

right_table=soup.find('table', {"class" : 'wikitable sortable'})
#print(right_table)

# Transfer to a DataFrame 
df = pd.read_html(str(right_table))[0]
df_all = pd.DataFrame(df)
#print(df_all.shape)
#df_all.head(5)

#### Get the dataframe after filtering out "Not assigned" boroughs and neighborhoods etc., restricting in the postal codes selected, and concatenating the neighborhoods under the same borough/postal code  

In [3]:
# Data shaping 

def concatenate_list_data(list):
    result= ''
    for element in list:
        result += str(element) +","
    return result[:-1]
#print(concatenate_list_data(['a','c']))

df.drop(  df[ df['Borough']=='Not assigned'].index, inplace=True)
restr = df['Neighborhood']=='Not assigned'
df.replace(to_replace=list(df[restr]['Neighborhood']), \
                 value= list(df[restr]['Borough']), inplace=True)
df_filtered = df
print("Size for filtered data table: ", df_filtered.shape)

post_lst=['M5G','M2H', 'M4B','M1J','M4G','M4M', 'M1R','M9V','M9L','M5V','M1B','M5A']

# print(df[df['Postcode'].isin(post_lst)])


newdf=pd.DataFrame(columns=['Postcode','Borough','Neighborhood'])

for pst in post_lst:
    restr= df['Postcode']==pst
    neigh=df[ restr]['Neighborhood'].to_list()
    neigh=concatenate_list_data(neigh)
    boro=df[restr]['Borough'].to_list()[0]
    addrows={'Postcode':pst,'Borough':boro,'Neighborhood':neigh}
    newdf=newdf.append(addrows, ignore_index=True)

#print("Size for the selected 12 postalcode data: ", newdf.shape)
newdf

Size for filtered data table:  (210, 3)


,Postcode,Borough,Neighborhood
0,M5G,Downtown Toronto,Central Bay Street
1,M2H,North York,Hillcrest Village
2,M4B,East York,"Woodbine Gardens,Parkview Hill"
3,M1J,Scarborough,Scarborough Village
4,M4G,East York,Leaside
5,M4M,East Toronto,Studio District
6,M1R,Scarborough,"Maryvale,Wexford"
7,M9V,Etobicoke,"Albion Gardens,Beaumond Heights,Humbergate,Jam..."
8,M9L,North York,Humber Summit
9,M5V,Downtown Toronto,"CN Tower,Bathurst Quay,Island airport,Harbourf..."


In [6]:
# Shape method
print("Size for all the data table=", df_all.shape)

print("Size for filtered data table: ", df_filtered.shape)
print('This dataframe has {} boroughs and {} neighborhoods.'.format(len(df_filtered['Borough'].unique()), df_filtered.shape[0] ))

print("Size for the selected 12 postalcode data: ", newdf.shape)
print('The dataframe has {} boroughs and {} sets of neighborhoods.'.format(len(newdf['Borough'].unique()), newdf.shape[0] ))

Size for all the data table= (287, 3)
Size for filtered data table:  (210, 3)
This dataframe has 11 boroughs and 210 neighborhoods.
Size for the selected 12 postalcode data:  (12, 3)
The dataframe has 6 boroughs and 12 sets of neighborhoods.


# Thanks
# End of Part 1